In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

In [3]:
chl = xr.open_dataset('../data/member_001/Chl_2D_mon_CESM001_1x1_198201-201701.nc')

mld = xr.open_dataset('member_001/MLD_2D_mon_CESM001_1x1_198201-201701.nc')

sss = xr.open_dataset('member_001/SSS_2D_mon_CESM001_1x1_198201-201701.nc')

sst = xr.open_dataset('member_001/SST_2D_mon_CESM001_1x1_198201-201701.nc')

u10 = xr.open_dataset('member_001/U10_2D_mon_CESM001_1x1_198201-201701.nc')

fg_co2= xr.open_dataset('member_001/FG-CO2_2D_mon_CESM001_1x1_198201-201701.nc')

xco2 = xr.open_dataset('member_001/XCO2_1D_mon_CESM001_native_198201-201701.nc')

icefrac = xr.open_dataset('member_001/iceFrac_2D_mon_CESM001_1x1_198201-201701.nc')

patm = xr.open_dataset('member_001/pATM_2D_mon_CESM001_1x1_198201-201701.nc')

pco2 = xr.open_dataset('member_001/pCO2_2D_mon_CESM001_1x1_198201-201701.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/shaunkim/Desktop/Columbia/GentineLab/ocean_co2/notebooks/member_001/MLD_2D_mon_CESM001_1x1_198201-201701.nc'

In [219]:
rmses=[]
names=['chl','mld','sss','sst','u10','icefrac','patm','xco2']
importances=pd.DataFrame(columns=names)
residuals=pd.DataFrame()

In [220]:
data_lat=chl.Chl.coords["ylat"].data
data_lon=chl.Chl.coords["xlon"].data

first_lat,first_lon=[],[]

for i in data_lon:
    for j in data_lat:
        first_lon.append(i)
        first_lat.append(j)
    
first_lon=np.array(first_lon)
first_lat=np.array(first_lat)

first_pco2=pco2.pCO2.data[0].flatten()

df_from_arr = pd.DataFrame(data=[first_pco2,first_lon,first_lat]).T
df_from_arr.columns=['pco2','xlon','ylat']
df_from_arr.dropna(subset = ["pco2"], inplace=True)


lon=df_from_arr["xlon"]
lat=df_from_arr["ylat"]

residuals["xlon"]=lon
residuals["ylat"]=lat


In [221]:
for i in range(421):
    print("no. ",i+1," month in progress")
    print(chl.Chl.time[i].data)
    train_chl=chl.Chl_socat.data[i].flatten()
    train_mld=mld.MLD_socat.data[i].flatten()
    train_sss=sss.SSS_socat.data[i].flatten()
    train_sst=sst.SST_socat.data[i].flatten()
    train_u10=u10.U10_socat.data[i].flatten()
    train_icefrac=icefrac.iceFrac_socat.data[i].flatten()
    train_patm=patm.pATM_socat.data[i].flatten()
    train_xco2=np.repeat(xco2.XCO2[i].data,len(train_chl))
    train_pco2=pco2.pCO2_socat.data[i].flatten()

    df_from_arr = pd.DataFrame(data=[train_chl,train_mld,train_sss,train_sst,train_u10,train_icefrac,train_patm,train_xco2,train_pco2]).T
    df_from_arr.columns=['chl','mld','sss','sst','u10','icefrac','patm','xco2','pco2']
    df_from_arr.dropna(subset = ["pco2"], inplace=True)
    
    X_train=df_from_arr.iloc[:,:-1]
    y_train=df_from_arr.iloc[:,-1:].values.ravel()
    
    
    regressor=RandomForestRegressor(n_estimators=10, random_state=42)
    regressor.fit(X_train, y_train)
    y_pred=regressor.predict(X_train)
    train_rmse=mean_squared_error(y_train, y_pred,squared=False)
    print("train rmse:", train_rmse)
    
    test_chl=chl.Chl.data[i].flatten()
    test_mld=mld.MLD.data[i].flatten()
    test_sss=sss.SSS.data[i].flatten()
    test_sst=sst.SST.data[i].flatten()
    test_u10=u10.U10.data[i].flatten()
    test_icefrac=icefrac.iceFrac.data[i].flatten()
    test_patm=patm.pATM.data[i].flatten()
    test_xco2=np.repeat(xco2.XCO2[i].data,len(train_chl))
    test_pco2=pco2.pCO2.data[i].flatten()
    
    
    df_from_arr2 = pd.DataFrame(data=[test_chl,test_mld,test_sss,test_sst,test_u10,test_icefrac,test_patm,test_xco2,test_pco2]).T
    df_from_arr2.columns=['chl','mld','sss','sst','u10','icefrac','patm','xco2','pco2']
    df_from_arr2.dropna(subset = ["pco2"], inplace=True)
    
    X_test=df_from_arr2.iloc[:,:-1]
    y_test=df_from_arr2.iloc[:,-1:].values.ravel()
    
    y_pred=regressor.predict(X_test)
    test_rmse=mean_squared_error(y_test, y_pred,squared=False)
    error=y_test-y_pred
    name=str(chl.Chl.time[i].data)
    
    residuals[name]=error
    print("test rmse:", test_rmse)
    rmses.append(test_rmse)
    print("\n")
    
    tmp=pd.DataFrame(regressor.feature_importances_).T
    tmp.columns=names
    importances=importances.append(tmp)
    if i==2:
        break


no.  1  month in progress
1982-01-01T00:00:00.000000000
train rmse: 0.031243432991327053
test rmse: 104.32652658877375


no.  2  month in progress
1982-02-01T00:00:00.000000000
train rmse: 0.09944672454355329
test rmse: 50.26048733308045


no.  3  month in progress
1982-03-01T00:00:00.000000000
train rmse: 0.06711227206357372
test rmse: 42.66619322369656




In [224]:
idx = pd.MultiIndex.from_product([data_lon,data_lat], names=["xlon","ylat"])
residuals=residuals.set_index(['xlon','ylat'])


KeyError: "None of ['xlon', 'ylat'] are in the columns"

In [225]:
residuals=residuals.reindex(idx)

In [227]:
residuals.to_xarray().hvplot()


:Image   [ylat,xlon]   (1982-01-01T00:00:00.000000000)

In [226]:
residuals

1982-01-01T00:00:00.000000000  1982-02-01T00:00:00.000000000  \
xlon  ylat                                                                  
0.5   -89.5                            NaN                            NaN   
      -88.5                            NaN                            NaN   
      -87.5                            NaN                            NaN   
      -86.5                            NaN                            NaN   
      -85.5                            NaN                            NaN   
...                                    ...                            ...   
359.5  85.5                     173.270063                      55.214692   
       86.5                     173.272747                      55.222022   
       87.5                     173.275389                      55.229199   
       88.5                     173.278006                      55.236248   
       89.5                     173.280621                      55.243222   

             1982-03-01T00:00:00.000000000  
xlon  ylat                                  
0.5   -89.5                            NaN  
      -88.5                            NaN  
      -87.5                            NaN  
      -86.5                            NaN  
      -85.5                            NaN  
...                                    ...  
359.5  85.5                      25.513367  
       86.5                      25.519391  
       87.5                      25.525363  
       88.5                      25.531363  
       89.5                      25.537312  

[64800 rows x 3 columns]

In [214]:
residuals.to_xarray()

<xarray.Dataset>
Dimensions:                        (xlon: 360, ylat: 180)
Coordinates:
  * xlon                           (xlon) float64 0.5 1.5 2.5 ... 358.5 359.5
  * ylat                           (ylat) float64 -89.5 -88.5 ... 88.5 89.5
Data variables:
    1982-01-01T00:00:00.000000000  (xlon, ylat) float64 nan nan ... 173.3 173.3

array([-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5, -81.5,
       -80.5, -79.5, -78.5, -77.5, -76.5, -75.5, -74.5, -73.5, -72.5,
       -71.5, -70.5, -69.5, -68.5, -67.5, -66.5, -65.5, -64.5, -63.5,
       -62.5, -61.5, -60.5, -59.5, -58.5, -57.5, -56.5, -55.5, -54.5,
       -53.5, -52.5, -51.5, -50.5, -49.5, -48.5, -47.5, -46.5, -45.5,
       -44.5, -43.5, -42.5, -41.5, -40.5, -39.5, -38.5, -37.5, -36.5,
       -35.5, -34.5, -33.5, -32.5, -31.5, -30.5, -29.5, -28.5, -27.5,
       -26.5, -25.5, -24.5, -23.5, -22.5, -21.5, -20.5, -19.5, -18.5,
       -17.5, -16.5, -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,  -9.5,
        -8.5,  -7.5,  -6.5,  -5.5,  -4.5,  -3.5,  -2.5,  -1.5,  -0.5,
         0.5,   1.5,   2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,
         9.5,  10.5,  11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,
        18.5,  19.5,  20.5,  21.5,  22.5,  23.5,  24.5,  25.5,  26.5,
        27.5,  28.5,  29.5,  30.5,  31.5,  32.5,  33.5,  34.5,  35.5,
        36.5,  37.5,